In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %cd /content/drive/MyDrive/cours/iasd-deep-golois/project/
# !pip install pybind11
# !chmod 777 ./compile.sh
# !./compile.sh


In [ ]:
# !cp golois.cpython-36m-x86_64-linux-gnu.so golois.so

# Best solution inspired from "Mobile Networks for Computer Go" 




In [1]:
!pwd

/tf/project


In [4]:
import os
import math
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras import layers 
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Input, Conv2D, DepthwiseConv2D, Dense, Concatenate, Add, ReLU, BatchNormalization, AvgPool2D, MaxPool2D, GlobalAveragePooling2D, Reshape, Permute, Lambda, Flatten, Activation
import golois

planes = 21
moves = 361
N = 10000
# epochs = 500
epochs = 50
batch=64 
filters = 64
trunk = filters

input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype ('float32')

print(input_data.ndim)
print(input_data.shape)

policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical (policy)

value = np.random.randint(2, size=(N,))
value = value.astype ('float32')

end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')

groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')

print ("getValidation", flush = True)
golois.getValidation (input_data, policy, value, end)

def bottleneck_block_mobilenet(x, expand=512, squeeze=128):
# def bottleneck_block_mobilenet(x, expand=96, squeeze=16):
    m = layers.Conv2D(expand, (1,1), kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(x)
    m = layers.BatchNormalization()(m)
    m = layers.Activation('relu')(m)
    m = layers.DepthwiseConv2D((3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(m)
    m = layers.BatchNormalization()(m)
    m = layers.Activation('relu')(m)
    m = layers.Conv2D(squeeze, (1,1), kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(m)
    m = layers.BatchNormalization()(m)
    return layers.Add()([m, x])

def bottleneck_block_shufflenet(tensor, expand=512, squeeze=128):
# def bottleneck_block_shufflenet(tensor, expand, squeeze=64):
    x = gconv(tensor, channels=expand, groups=4)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = channel_shuffle(x, groups=4)
    x = DepthwiseConv2D(kernel_size=3, padding='same', kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(x)
    x = BatchNormalization()(x)

    x = gconv(x, channels=squeeze, groups=4)
    x = BatchNormalization()(x)

    x = Add()([tensor, x])
    output = ReLU()(x)
    return output

def gconv(tensor, channels, groups):
    input_ch = tensor.get_shape().as_list()[-1]
    group_ch = input_ch // groups
    output_ch = channels // groups
    groups_list = []

    for i in range(groups):
        group_tensor = tensor[:, :, :, i * group_ch: (i+1) * group_ch]
        group_tensor = Conv2D(output_ch, 1)(group_tensor)
        groups_list.append(group_tensor)

    output = Concatenate()(groups_list)
    return output

def channel_shuffle(x, groups):  
    _, width, height, channels = x.get_shape().as_list()
    group_ch = channels // groups

    x = Reshape([width, height, group_ch, groups])(x)
    x = Permute([1, 2, 4, 3])(x)
    x = Reshape([width, height, channels])(x)
    return x

4
(10000, 19, 19, 21)
getValidation


In [6]:
input = keras.Input(shape=(19, 19, planes), name='board')

x = layers.Conv2D(trunk, 1, padding='same',kernel_regularizer=regularizers.l2(0.0001))(input)
x_bifurcation = BatchNormalization()(x)

blocks = 28 # 33
filters = 174 # 198
x = ReLU()(x_bifurcation)
for i in range (blocks):
    x = bottleneck_block_mobilenet(x, filters, trunk)
    # Is is also possible to alternate one of each types : one mobilenet followed by one sufflenet
    # x = bottleneck_block_shufflenet(x, filters, trunk)

# In series
for i in range (blocks):
    x = bottleneck_block_shufflenet(x, filters, trunk)

# In parallel
# y = ReLU()(x_bifurcation)
# for i in range (blocks):
#    y = bottleneck_block_shufflenet(y, filters, trunk)
# concatenate
# x = layers.concatenate([x, y], axis=-1)

# fully convolutional, no dense layer
policy_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False
                            , kernel_regularizer=regularizers.l2(0.0001))(x)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Activation('softmax', name='policy')(policy_head)

value_head = layers.GlobalAveragePooling2D()(x)
value_head = layers.Dense(50, activation='relu',kernel_regularizer=regularizers.l2(0.0001))(value_head)
value_head = layers.Dense(1, activation='sigmoid', name='value'
                          ,kernel_regularizer=regularizers.l2(0.0001))(value_head)

model = keras.Model(inputs=input, outputs=[policy_head, value_head])

model.summary ()

# 2nd Train momentum 0.8; 3rd Train momentum 0.9
# model.compile(optimizer=keras.optimizers.SGD(lr=0.000001, momentum=0.9,nesterov=False),
# 1st Train
#model.compile(optimizer=keras.optimizers.SGD(lr=0.005, momentum=0.9),
#              loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
#              # 2nd Train
#              loss_weights={'policy' : 2.0, 'value' : 1.0},
#              # loss_weights={'policy' : 1.0, 'value' : 1.0},
#              metrics={'policy': 'categorical_accuracy', 'value': 'mse'})

# In parallel
# Different optimizers and loss weighting schemes were explored:
# model.compile(optimizer=keras.optimizers.SGD(lr=0.05, momentum=0.9),
# model.compile(optimizer=keras.optimizers.Adam(lr=0.001, momentum=0.9),
# model.compile(optimizer=keras.optimizers.SGD(lr=0.000001, momentum=0.9,nesterov=False),
model.compile(optimizer=keras.optimizers.Adagrad(lr=0.1),
              loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
              # 2nd Train
              # loss_weights={'policy' : 2.0, 'value' : 1.0},
              loss_weights={'policy' : 1.0, 'value' : 1.0},
              metrics={'policy': 'categorical_accuracy', 'value': 'mse'})
            

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
board (InputLayer)              [(None, 19, 19, 21)] 0                                            
__________________________________________________________________________________________________
conv2d_564 (Conv2D)             (None, 19, 19, 64)   1408        board[0][0]                      
__________________________________________________________________________________________________
batch_normalization_338 (BatchN (None, 19, 19, 64)   256         conv2d_564[0][0]                 
__________________________________________________________________________________________________
re_lu_114 (ReLU)                (None, 19, 19, 64)   0           batch_normalization_338[0][0]    
____________________________________________________________________________________________

__________________________________________________________________________________________________
tf.__operators__.getitem_604 (S (None, 19, 19, 43)   0           batch_normalization_481[0][0]    
__________________________________________________________________________________________________
tf.__operators__.getitem_605 (S (None, 19, 19, 43)   0           batch_normalization_481[0][0]    
__________________________________________________________________________________________________
tf.__operators__.getitem_606 (S (None, 19, 19, 43)   0           batch_normalization_481[0][0]    
__________________________________________________________________________________________________
tf.__operators__.getitem_607 (S (None, 19, 19, 43)   0           batch_normalization_481[0][0]    
__________________________________________________________________________________________________
conv2d_777 (Conv2D)             (None, 19, 19, 16)   704         tf.__operators__.getitem_604[0][0
__________

In [7]:
#!cp ./test-12052021_0829_last.h5 ./test-12052021_0829_previous.h5

# 2nd and 3rd Train
# model.load_weights("./test-14052021_last.h5")

# learning rate schedule
# def step_decay(epoch):
# 	initial_lrate = 0.000005 # 0.005
# 	drop = 0.1
# 	epochs_drop = 2 # 10.0
# 	lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
# 	return lrate

# def lr_scheduler(epoch):
#  if epoch < 100:
#    return 0.00005
#  elif epoch < 150:
#    return 0.000025
#  elif epoch >= 150:
#    return 0.00001

# checkpoint_path = "training_1/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                 save_weights_only=True,
#                                                 verbose=1)

# adapt LR as epochs grows
#lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

# 1st model obtained using : 
# lr_callback = tf.keras.callbacks.LearningRateScheduler(step_decay)


# checkpoint_path = '/checkpoint'
checkpoint_path = "checkpoint/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


callbacks_list = [
    tf.keras.callbacks.EarlyStopping(
        monitor='policy_categorical_accuracy', patience = 10, verbose = 1, mode = "auto",
        restore_best_weights = True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        save_weights_only=True,
        monitor='loss',
        mode='min',
        save_best_only=True,
        verbose =1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='loss',
        # 1st Train
        # factor = 0.2,
        # patience = 10
        # 2nd Train
        # factor = 0.2,
        # patience =  10       
        # 3rd Train
        factor = 0.2,
        patience = 3
    ),
    keras.callbacks.TensorBoard(
        log_dir='/tf/my_log_dir',
        histogram_freq=5,
        embeddings_freq=0
    )
]

for i in range (1, epochs + 1):
    print ('epoch ' + str (i))
    golois.getBatch (input_data, policy, value, end, groups)
    history = model.fit(input_data,
                        {'policy': policy, 'value': value}, 
                        epochs=1, 
                        callbacks=callbacks_list,
                        batch_size=batch)
    if (i % 10 == 0):
        golois.getValidation (input_data, policy, value, end)
        val = model.evaluate (input_data,
                              [policy, value], verbose = 0, batch_size=batch)
        print ("val =", val)
    
    if (i %100 == 0):
            model.save ('/tf/my_log_dir/run_20210817_Adagrad_Parallel.h5')

# 1st Train
# model.save ('test-14052021_last.h5')
# 2nd Train
model.save ('run_20210817_Adagrad_Parallel.h5')



epoch 1
157/157 [==============================] - 58s 207ms/step - loss: 8.0601 - policy_loss: 6.2255 - value_loss: 1.2357 - policy_categorical_accuracy: 4.9348e-04 - value_mse: 0.2667

Epoch 00001: loss improved from inf to 7.36175, saving model to checkpoint/cp.ckpt
epoch 2
157/157 [==============================] - 32s 205ms/step - loss: 7.1705 - policy_loss: 5.8889 - value_loss: 0.6933 - policy_categorical_accuracy: 0.0021 - value_mse: 0.2501

Epoch 00001: loss improved from 7.36175 to 7.17052, saving model to checkpoint/cp.ckpt
epoch 3
157/157 [==============================] - 32s 206ms/step - loss: 7.1600 - policy_loss: 5.8889 - value_loss: 0.6932 - policy_categorical_accuracy: 0.0015 - value_mse: 0.2500

Epoch 00001: loss improved from 7.17052 to 7.15999, saving model to checkpoint/cp.ckpt
epoch 4
157/157 [==============================] - 33s 207ms/step - loss: 7.1495 - policy_loss: 5.8889 - value_loss: 0.6931 - policy_categorical_accuracy: 0.0010 - value_mse: 0.2500

Epoch 0

157/157 [==============================] - 32s 206ms/step - loss: 6.9393 - policy_loss: 5.8889 - value_loss: 0.6929 - policy_categorical_accuracy: 0.0016 - value_mse: 0.2499

Epoch 00001: loss improved from 6.94570 to 6.93927, saving model to checkpoint/cp.ckpt
val = [6.9362053871154785, 5.888870716094971, 0.6929903626441956, 0.0010000000474974513, 0.24992138147354126]
epoch 31
157/157 [==============================] - 33s 208ms/step - loss: 6.9334 - policy_loss: 5.8889 - value_loss: 0.6933 - policy_categorical_accuracy: 0.0010 - value_mse: 0.2501

Epoch 00001: loss improved from 6.93927 to 6.93341, saving model to checkpoint/cp.ckpt
epoch 32
157/157 [==============================] - 33s 207ms/step - loss: 6.9273 - policy_loss: 5.8889 - value_loss: 0.6932 - policy_categorical_accuracy: 0.0016 - value_mse: 0.2500

Epoch 00001: loss improved from 6.93341 to 6.92727, saving model to checkpoint/cp.ckpt
epoch 33
157/157 [==============================] - 32s 206ms/step - loss: 6.9212 - po

In [9]:
!ls

Board.h					golois.so
Game.h					importGolois.ipynb
Golois.ipynb				importGolois_Thomas.ipynb
README					importGolois_Thomas_08052021.ipynb
Rzone.h					model-07052021.h5
backup_tmp.ipynb			test-08052021.h5
compile.sh				test-12052021_0823.h5
compileMAC.sh				test-12052021_0829_MobileNetOnly.h5
core.149472				test-12052021_0829_last.h5
games.data				test-12052021_0829_previous.h5
golois.cpp				test.h5
golois.cpython-36m-x86_64-linux-gnu.so	training_1
golois.cpython-38-x86_64-linux-gnu.so	validation.data
golois.py
